## Create RAG ratings for the aggregate data

## Set-up

### Packages and file paths

In [1]:
# Import required packages
from dataclasses import dataclass
import math
import numpy as np
import os
import pandas as pd
#from statsmodels.stats.weightstats import DescrStatsW

# Set display options
pd.set_option('display.max_rows', 100)

In [2]:
# File paths
@dataclass(frozen=True)
class Paths:
    '''Paths to data and files'''
    survey = '../data/survey_data'
    aggregate = 'aggregate_scores.csv'
    rag = 'aggregate_scores_rag.csv'


paths = Paths()

## Import aggregate scores

In [3]:
data = pd.read_csv(os.path.join(paths.survey, paths.aggregate))

# Preview dataframe
data.tail()

,variable,mean,count,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab
2011,wealth_score,0.266667,30.0,School G,All,All,All,Non-SEN
2012,future_score,7.823529,17.0,School G,All,All,All,Non-SEN
2013,climate_score,2.224490,49.0,School G,All,All,All,Non-SEN
2014,social_score,11.904762,42.0,School G,All,All,All,Non-SEN
2015,bully_score,7.423077,52.0,School G,All,All,All,Non-SEN


## Find weighted mean and SD within each group

Define function for calculation weighted mean and weighted standard deviation.

In [4]:
def descriptives(values, counts):
    '''
    This function uses school-level data. Using the mean and count from each
    group, it calculates the weighted mean and weighted standard deviation
    of scores across the groups. It returns this, alongside a count of the
    pupils and groups included.

    Additional information about weighted standard deviation:
    This normalises weights so they sum 1 (and so they can't all be 0).
    It returns the biased variance and is like a weighted version of np.std().
    For small samples, may want to alter to unbiased variance.
    Based on: https://stackoverflow.com/questions/2413522/weighted-standard-deviation-in-numpy

    Parameters
    ----------
    values : pandas series
        Dataframe column with the mean scores in each group, NaN removed
    counts: pandas series
        Dataframe column with the count of pupils in each group, NaN removed

    Returns
    -------
    result : pandas Series
        Series with each of the calculations, where index if the name of the
        calculation
    '''
    # Check for NaN
    if values.isnull().any():
        raise ValueError('There must be no NaN in the values column.')
    if counts.isnull().any():
        raise ValueError('There must be no NaN in the counts column.')

    # Weighted mean
    average = np.average(values, weights=counts)
    # Weighted std
    variance = np.average((values-average)**2, weights=counts)
    std = math.sqrt(variance)

    # Total sample size
    n_pupils = counts.sum(skipna=True)
    # Total number of groups
    n_groups = counts.count()

    # Combine into a series
    result = pd.Series(
        [n_pupils, n_groups, average, std],
        index=['total_pupils', 'group_n', 'group_wt_mean', 'group_wt_std'])
    return(result)

Apply function to each of the groups.

In [5]:
# Define the variables to simultaneously group by (remember, as this is working
# on the aggregated data, it will just have possible combinations between
# the filters of "All" for every characteristic, or "All" for every except one)
groups = ['variable', 'year_group_lab', 'gender_lab', 'fsm_lab', 'sen_lab']

# Filter to non-nan rows (as other rows can't/won't be used in calculation)
non_nan = data[~(data['mean'].isnull()) & ~(data['count'].isnull())]

# Groupby variable and all the characteristics, and return
wt_mean = (non_nan
           .groupby(groups)
           .apply(lambda x: descriptives(x['mean'], x['count']))
           .reset_index())

# Preview the dataframe
display(wt_mean.head())
display(wt_mean.tail())

,variable,year_group_lab,gender_lab,fsm_lab,sen_lab,total_pupils,group_n,group_wt_mean,group_wt_std
0,accept_score,All,All,All,All,693.0,7.0,10.060606,0.282770
1,accept_score,All,All,All,Non-SEN,393.0,7.0,10.055980,0.348867
2,accept_score,All,All,All,SEN,264.0,6.0,10.030303,0.358692
3,accept_score,All,All,FSM,All,334.0,7.0,9.976048,0.348463
4,accept_score,All,All,Non-FSM,All,282.0,7.0,10.109929,0.357952


,variable,year_group_lab,gender_lab,fsm_lab,sen_lab,total_pupils,group_n,group_wt_mean,group_wt_std
281,wellbeing_score,All,All,Non-FSM,All,209.0,7.0,21.272727,0.598172
282,wellbeing_score,All,Boy,All,All,73.0,5.0,20.972603,1.176615
283,wellbeing_score,All,Girl,All,All,65.0,5.0,20.076923,0.997094
284,wellbeing_score,Year 10,All,All,All,219.0,6.0,21.041096,0.650181
285,wellbeing_score,Year 8,All,All,All,270.0,7.0,21.000000,0.767547


Add the calculated weighted means and SDs back to the school-level results.

In [6]:
# Add the record of the weighted mean and STD back to the school-level results
rag = pd.merge(data, wt_mean, how='left', on=groups)

# Preview the dataframe
display(rag.head())
display(rag.tail())

,variable,mean,count,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,total_pupils,group_n,group_wt_mean,group_wt_std
0,autonomy_score,17.865854,82.0,School A,All,All,All,All,487.0,7.0,17.831622,0.297966
1,life_satisfaction_score,5.105263,114.0,School A,All,All,All,All,725.0,7.0,5.055172,0.329480
2,optimism_score,12.025210,119.0,School A,All,All,All,All,729.0,7.0,12.065844,0.261534
3,wellbeing_score,21.151163,86.0,School A,All,All,All,All,538.0,7.0,20.964684,0.452018
4,esteem_score,12.739726,73.0,School A,All,All,All,All,476.0,7.0,12.689076,0.379053


,variable,mean,count,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,total_pupils,group_n,group_wt_mean,group_wt_std
2011,wealth_score,0.266667,30.0,School G,All,All,All,Non-SEN,301.0,7.0,0.368771,0.085103
2012,future_score,7.823529,17.0,School G,All,All,All,Non-SEN,172.0,7.0,7.415698,0.437618
2013,climate_score,2.224490,49.0,School G,All,All,All,Non-SEN,434.0,7.0,2.532258,0.158200
2014,social_score,11.904762,42.0,School G,All,All,All,Non-SEN,301.0,7.0,11.823920,0.293917
2015,bully_score,7.423077,52.0,School G,All,All,All,Non-SEN,430.0,7.0,7.644186,0.098142


## Create RAG column based on whether 1SD above or below

All scores are ordered in the positive direction so for every score, a higher result means you are "above average" and a "better outcome", and vice versa for lower scores.

In [7]:
# Find 1 SD above and below mean
rag['lower'] = rag['group_wt_mean'] - rag['group_wt_std']
rag['upper'] = rag['group_wt_mean'] + rag['group_wt_std']

In [8]:
# Create RAG column
conditions = [(rag['mean'] <= rag['lower']),
              (rag['mean'] > rag['lower']) & (rag['mean'] < rag['upper']),
              (rag['mean'] >= rag['upper'])]
choices = ['below', 'average', 'above']
rag.loc[:, 'rag'] = np.select(conditions, choices, default=np.nan)

 Show some examples

In [9]:
# Autonomy score of all pupils at each school
test = rag[
    (rag['variable'] == 'autonomy_score') &
    (rag['year_group_lab'] == 'All') &
    (rag['gender_lab'] == 'All') &
    (rag['fsm_lab'] == 'All') &
    (rag['sen_lab'] == 'All')]
display(test)

# Acceptance score of Year 10 pupils at each school
test = rag[
    (rag['variable'] == 'accept_score') &
    (rag['year_group_lab'] == 'Year 10') &
    (rag['gender_lab'] == 'All') &
    (rag['fsm_lab'] == 'All') &
    (rag['sen_lab'] == 'All')]
display(test)

,variable,mean,count,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,total_pupils,group_n,group_wt_mean,group_wt_std,lower,upper,rag
0,autonomy_score,17.865854,82.0,School A,All,All,All,All,487.0,7.0,17.831622,0.297966,17.533657,18.129588,average
288,autonomy_score,17.878378,74.0,School B,All,All,All,All,487.0,7.0,17.831622,0.297966,17.533657,18.129588,average
576,autonomy_score,18.181818,77.0,School C,All,All,All,All,487.0,7.0,17.831622,0.297966,17.533657,18.129588,above
864,autonomy_score,17.810345,58.0,School D,All,All,All,All,487.0,7.0,17.831622,0.297966,17.533657,18.129588,average
1152,autonomy_score,17.907692,65.0,School E,All,All,All,All,487.0,7.0,17.831622,0.297966,17.533657,18.129588,average
1440,autonomy_score,17.136364,66.0,School F,All,All,All,All,487.0,7.0,17.831622,0.297966,17.533657,18.129588,below
1728,autonomy_score,17.969231,65.0,School G,All,All,All,All,487.0,7.0,17.831622,0.297966,17.533657,18.129588,average


,variable,mean,count,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,total_pupils,group_n,group_wt_mean,group_wt_std,lower,upper,rag
83,accept_score,10.047619,42.0,School A,Year 10,All,All,All,275.0,6.0,10.032727,0.314811,9.717916,10.347538,average
371,accept_score,9.976744,43.0,School B,Year 10,All,All,All,275.0,6.0,10.032727,0.314811,9.717916,10.347538,average
659,accept_score,10.155172,58.0,School C,Year 10,All,All,All,275.0,6.0,10.032727,0.314811,9.717916,10.347538,average
947,accept_score,9.463415,41.0,School D,Year 10,All,All,All,275.0,6.0,10.032727,0.314811,9.717916,10.347538,below
1235,accept_score,9.940000,50.0,School E,Year 10,All,All,All,275.0,6.0,10.032727,0.314811,9.717916,10.347538,average
1523,accept_score,NaN,NaN,School F,Year 10,All,All,All,275.0,6.0,10.032727,0.314811,9.717916,10.347538,nan
1811,accept_score,10.585366,41.0,School G,Year 10,All,All,All,275.0,6.0,10.032727,0.314811,9.717916,10.347538,above


## Add topic names

For example, "Autonomy" for "autonomy_score".

In [10]:
# Define the labels for each topic
var_lab = {
    'autonomy_score': 'Autonomy',
    'life_satisfaction_score': 'Life satisfaction',
    'optimism_score': 'Optimism',
    'wellbeing_score': 'Psychological wellbeing',
    'esteem_score': 'Self-esteem',
    'stress_score': 'Stress and coping',
    'appearance_score': 'Feelings around appearance',
    'negative_score': 'Negative affect',
    'lonely_score': 'Loneliness',
    'support_score': 'Supporting own wellbeing',
    'sleep_score': 'Sleep',
    'physical_score': 'Physical activity',
    'free_like_score': 'Free time',
    'media_score': 'Social media use',
    'places_score': 'Places to go and things to do',
    'talk_score': 'Talking about feelings',
    'accept_score': 'Acceptance',
    'school_belong_score': 'School connection',
    'staff_relationship_score': 'Support from staff',
    'home_relationship_score': 'Support from parents/carers',
    'home_happy_score': 'Home environment',
    'local_env_score': 'Local environment',
    'discrim_score': 'Discrimination',
    'belong_local_score': 'Local connection',
    'wealth_score': 'Relative wealth',
    'future_score': 'Future opportunities',
    'climate_score': 'Climate change',
    'social_score': 'Support from friends',
    'bully_score': 'Bullying'
}

# Add label column
rag['variable_lab'] = rag['variable'].map(var_lab)

# View the labels
rag[['variable', 'variable_lab']].drop_duplicates()

,variable,variable_lab
0,autonomy_score,Autonomy
1,life_satisfaction_score,Life satisfaction
2,optimism_score,Optimism
3,wellbeing_score,Psychological wellbeing
4,esteem_score,Self-esteem
5,stress_score,Stress and coping
6,appearance_score,Feelings around appearance
7,negative_score,Negative affect
8,lonely_score,Loneliness
9,support_score,Supporting own wellbeing


## Add descriptions

In [11]:
# Create dictionary with the description to accompany each score
describe = {
    'autonomy_score': '''How 'in control' young people feel of their life''',
    'life_satisfaction_score': 'How satisfied young people feel with their life',
    'optimism_score': '''Young people's hopefulness and confidence for the future''',
    'wellbeing_score': 'How positive and generally happy young people feel regarding their life',
    'esteem_score': 'How much young people value themselves',
    'stress_score': 'Managing stress levels and coping with difficulties',
    'appearance_score': '''Young people's feelings around the way that they look''',
    'negative_score': 'The frequency with which young people experience emotional difficulties',
    'lonely_score': 'How often young people feel lonely',
    'support_score': '''Young people's knowledge on supporting themselves and looking for advice''',
    'sleep_score': 'How much sleep young people get',
    'physical_score': 'How physically active young people are',
    'free_like_score': 'How often young people can do things that they like in their free time',
    'media_score': 'How much time young people spend on social media',
    'places_score': '''Whether young people feel there are places to go and things to do in their free time''',
    'talk_score': '''How positively/negatively young people feel about talking with others about feeling down''',
    'accept_score': 'Whether young people feel accepted by different groups of people in their life',
    'school_belong_score': 'Feelings of belonging at school',
    'staff_relationship_score': 'The support received from adults at school',
    'home_relationship_score': 'The support received from adults at home',
    'home_happy_score': '''Young people's feelings regarding the home that they live in''',
    'local_env_score': 'How young people feel regarding the area where they live',
    'discrim_score': 'Whether young people feel discriminated against',
    'belong_local_score': '''Young people's feelings of belonging in their local area''',
    'wealth_score': 'Whether young people feel their family is richer, poorer or the same as their friends',
    'future_score': 'How young people feel regarding the future options for work, education or training in their local area',
    'climate_score': 'Worries regarding climate change',
    'social_score': 'The support young people receive from their peers',
    'bully_score': 'The frequency with which young people experience different types of bullying'
}

# Add labels to dataframe
rag['description'] = rag['variable'].map(describe)

# View each of the topics and their label
rag[['variable_lab', 'description']].drop_duplicates()

,variable_lab,description
0,Autonomy,How 'in control' young people feel of their life
1,Life satisfaction,How satisfied young people feel with their life
2,Optimism,Young people's hopefulness and confidence for ...
3,Psychological wellbeing,How positive and generally happy young people ...
4,Self-esteem,How much young people value themselves
5,Stress and coping,Managing stress levels and coping with difficu...
6,Feelings around appearance,Young people's feelings around the way that th...
7,Negative affect,The frequency with which young people experien...
8,Loneliness,How often young people feel lonely
9,Supporting own wellbeing,Young people's knowledge on supporting themsel...


## Save as csv file

In [12]:
rag.to_csv(os.path.join(paths.survey, paths.rag), index=False)